# 03 · 合成注入訓練 · Logistic + Isotonic
建立（正/負）樣本 → 特徵 → 訓練 → 可靠度校準

In [ ]:
# 環境設定與依賴安裝（Colab）
import sys, subprocess, pkgutil

def pipi(*pkgs):
    """安裝套件的輔助函式"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *pkgs])

# 安裝必要套件（避免 numpy 2.0 相容性問題）
print("🚀 正在安裝依賴套件...")
try:
    import numpy as np
    import lightkurve as lk
    import sklearn
    print("✅ 基礎套件已安裝")
except Exception:
    pipi("numpy<2", "lightkurve", "astroquery", "scikit-learn", "matplotlib", 
         "wotan", "transitleastsquares", "joblib", "torch")
    print("✅ 依賴套件安裝完成")

# 檢查 GPU 資訊
import torch if 'torch' in [m.name for m in pkgutil.iter_modules()] else None
gpu_available = False
is_l4_gpu = False

if torch is not None and torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🖥️ GPU 型號: {gpu_name}")
    print(f"   記憶體: {gpu_memory:.2f} GB")
    gpu_available = True
    
    # 如果是 NVIDIA L4，提供 BF16 優化建議
    if "L4" in gpu_name:
        is_l4_gpu = True
        print("💡 偵測到 NVIDIA L4 GPU - 支援高效能 BF16 運算")
        print("   建議在訓練時使用 torch.autocast('cuda', dtype=torch.bfloat16)")
        print("\n   BF16 範例程式碼：")
        print("   ```python")
        print("   with torch.autocast('cuda', dtype=torch.bfloat16):")
        print("       output = model(input)")
        print("       loss = criterion(output, target)")
        print("   ```")
else:
    try:
        # 使用 nvidia-smi 檢查 GPU
        result = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], 
                              capture_output=True, text=True, check=False)
        if result.returncode == 0:
            gpu_name = result.stdout.strip()
            print(f"🖥️ GPU 型號: {gpu_name}")
            gpu_available = True
            if "L4" in gpu_name:
                is_l4_gpu = True
                print("💡 偵測到 NVIDIA L4 GPU - 支援高效能 BF16 運算")
    except:
        print("⚠️ 未偵測到 GPU，將使用 CPU 運算")

print("\n環境設定完成！")

In [ ]:
import lightkurve as lk, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import average_precision_score, precision_recall_curve
import json, os, joblib

# ========== 實際資料獲取 ==========
print("📡 從 NASA 資料庫獲取實際資料...")

# 方法 1: 使用確認的 TESS 行星作為正樣本
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive

try:
    # 獲取已確認的 TESS 凌日行星
    confirmed = NasaExoplanetArchive.query_criteria(
        table="ps",
        select="pl_name,tic_id,pl_orbper,pl_trandep",
        where="discoverymethod='Transit' and disc_facility like '%TESS%' and tic_id is not null",
        format="table"
    )
    print(f"✅ 找到 {len(confirmed)} 個確認的 TESS 行星")
    
    # 獲取 TOI 假陽性作為負樣本
    false_positives = NasaExoplanetArchive.query_criteria(
        table="toi",
        select="tid,tfopwg_disp,toi_period",
        where="tfopwg_disp='FP'",
        format="table"
    )
    print(f"✅ 找到 {len(false_positives)} 個 TOI 假陽性")
except Exception as e:
    print(f"⚠️ 無法連接 NASA Archive，使用模擬資料: {e}")
    confirmed = None
    false_positives = None

# ========== 建立訓練資料集 ==========
# 如果有真實資料，優先使用；否則使用合成資料
if confirmed is not None and len(confirmed) > 0:
    print("\n🎯 使用真實 TESS 資料訓練...")
    # 這裡簡化處理，實際應用需要下載完整光曲線
    # 以下仍使用合成資料示範，但參數基於真實行星
    real_periods = confirmed['pl_orbper'][confirmed['pl_orbper'].mask == False].data[:10]
    real_depths = confirmed['pl_trandep'][confirmed['pl_trandep'].mask == False].data[:10]
else:
    print("\n🔧 使用合成資料訓練（模擬真實參數分布）...")
    real_periods = None
    real_depths = None

# 下載一條基準光曲線
target = "TIC 25155310"  # WASP-121 b - 確認的熱木星
try:
    lc = lk.search_lightcurve(target, mission="TESS", author="SPOC").download().remove_nans()
    flat = lc.flatten(window_length=401)
    t = flat.time.value
    y = flat.flux.value
    print(f"✅ 成功下載 {target} 光曲線作為基底")
except:
    # 備用：生成模擬光曲線
    t = np.linspace(0, 27, 20000)  # 27天 TESS 觀測
    y = 1.0 + np.random.normal(0, 0.001, len(t))
    print("⚠️ 使用模擬光曲線")

# 建立資料集：注入正樣本 + 負樣本
rng = np.random.default_rng(42)

def inject_box(time, flux, period, depth, duration, t0):
    """注入箱型凌日信號"""
    model = flux.copy()
    phase = ((time - t0) % period) / period
    in_transit = (phase < (duration/period))
    model[in_transit] *= (1.0 - depth)
    return model

def bls_feats(time, flux):
    """提取 BLS 特徵"""
    try:
        bls = lk.LightCurve(time=time, flux=flux).to_periodogram(
            method="bls", minimum_period=0.5, maximum_period=20
        )
        return dict(
            bls_period=bls.period_at_max_power.value,
            bls_t0=bls.transit_time_at_max_power.value,
            bls_duration=bls.duration_at_max_power.value,
            bls_snr=bls.max_power.value
        )
    except:
        return dict(bls_period=1.0, bls_t0=0.0, bls_duration=0.1, bls_snr=0.0)

X, ylab = [], []

# 生成 200 個正樣本（使用真實參數分布）
print("\n生成訓練樣本...")
for i in range(200):
    if real_periods is not None and i < len(real_periods):
        # 使用真實行星參數
        P = float(real_periods[i])
        D = float(real_depths[i]) / 1e6 if real_depths is not None and i < len(real_depths) else rng.uniform(0.0005, 0.02)
    else:
        # 使用模擬參數（基於 TESS 行星統計）
        P = rng.uniform(0.6, 10)  # 典型 TESS 行星週期
        D = rng.uniform(0.0005, 0.02)  # 0.05%~2% 深度
    
    W = rng.uniform(0.02, 0.1)  # 週期比例持續時間
    T0 = t.min() + rng.uniform(0, P)
    fx = inject_box(t, y, P, D, W*P, T0)
    X.append(bls_feats(t, fx))
    ylab.append(1)

# 生成 200 個負樣本
for _ in range(200):
    # 負樣本：原曲線或加噪
    noise = rng.normal(0, np.std(y)*0.2, size=y.size)
    fx = y + noise
    X.append(bls_feats(t, fx))
    ylab.append(0)

df = pd.DataFrame(X)
df['label'] = ylab
print(f"✅ 生成 {len(df)} 個訓練樣本")
print(f"   正樣本（行星）: {(df['label']==1).sum()}")
print(f"   負樣本（非行星）: {(df['label']==0).sum()}")
print("\n特徵統計:")
print(df.describe())

In [ ]:
# 訓練 + 校準
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

X = df[['bls_period','bls_duration','bls_snr']].values
y = df['label'].values
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 基礎模型訓練（scikit-learn）
base = LogisticRegression(max_iter=200)
clf = CalibratedClassifierCV(base, method="isotonic", cv=3)
pipe = make_pipeline(StandardScaler(with_mean=False), clf)
pipe.fit(Xtr, ytr)
probs = pipe.predict_proba(Xte)[:,1]
ap = average_precision_score(yte, probs)
print(f"Average Precision Score: {ap:.3f}")

# 若使用 PyTorch 神經網路進行深度學習（範例）
# 在 NVIDIA L4 GPU 上使用 BF16 加速
if 'is_l4_gpu' in locals() and is_l4_gpu and 'torch' in sys.modules:
    import torch
    import torch.nn as nn
    
    print("\n🚀 PyTorch BF16 訓練範例（針對 NVIDIA L4 優化）:")
    
    class SimpleNN(nn.Module):
        def __init__(self, input_dim=3):
            super().__init__()
            self.fc1 = nn.Linear(input_dim, 64)
            self.fc2 = nn.Linear(64, 32)
            self.fc3 = nn.Linear(32, 1)
            self.relu = nn.ReLU()
            self.sigmoid = nn.Sigmoid()
        
        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.relu(self.fc2(x))
            x = self.sigmoid(self.fc3(x))
            return x
    
    # 示範 BF16 自動混合精度訓練
    model = SimpleNN().cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.BCELoss()
    
    # 轉換資料到 PyTorch tensors
    X_tensor = torch.FloatTensor(Xtr).cuda()
    y_tensor = torch.FloatTensor(ytr).unsqueeze(1).cuda()
    
    # 使用 BF16 autocast 進行訓練（NVIDIA L4 優化）
    print("使用 BF16 混合精度訓練...")
    model.train()
    for epoch in range(10):
        with torch.autocast('cuda', dtype=torch.bfloat16):
            outputs = model(X_tensor)
            loss = criterion(outputs, y_tensor)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 5 == 0:
            print(f"  Epoch {epoch}, Loss: {loss.item():.4f}")
    
    print("✅ BF16 訓練完成（使用 NVIDIA L4 硬體加速）")
else:
    print("\n📝 提示：若需要使用深度學習模型，可安裝 PyTorch 並在 GPU 上執行")

In [ ]:
# 可靠度曲線
from sklearn.calibration import calibration_curve
prob_true, prob_pred = calibration_curve(yte, probs, n_bins=10)
import matplotlib.pyplot as plt
plt.plot(prob_pred, prob_true, marker="o"); plt.plot([0,1],[0,1],'--'); plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Calibration");

## Performance Tips

### BF16 vs FP16 vs FP32 精度比較

當使用 GPU 進行深度學習訓練時，選擇適當的數值精度可以顯著影響訓練速度與模型精確度：

| 精度類型 | 位元數 | 優點 | 缺點 | 建議使用情境 |
|---------|-------|------|------|------------|
| **FP32** | 32-bit | • 最高精度<br>• 最佳數值穩定性 | • 記憶體使用量大<br>• 運算速度慢 | • 小型模型<br>• 需要最高精度的科學計算 |
| **FP16** | 16-bit | • 記憶體減半<br>• 2-4x 加速（支援硬體） | • 可能溢位/欠位<br>• 梯度消失風險 | • 影像分類<br>• 已調校良好的模型 |
| **BF16** | 16-bit | • 與 FP32 相同的指數範圍<br>• 更好的數值穩定性<br>• 2-4x 加速（L4/A100） | • 精度略低於 FP32<br>• 需要新硬體支援 | • **推薦用於 NVIDIA L4**<br>• 大型語言模型<br>• 天文資料處理 |

### NVIDIA L4 GPU 優化建議

NVIDIA L4 GPU 特別優化了 BF16 運算，在保持數值穩定性的同時提供顯著的效能提升：

```python
# BF16 自動混合精度訓練（推薦）
with torch.autocast('cuda', dtype=torch.bfloat16):
    outputs = model(inputs)
    loss = criterion(outputs, targets)
loss.backward()
optimizer.step()
```

### 何時改回 FP32

在以下情況建議改回 FP32 精度：

1. **最終推論階段**：當需要最精確的預測結果時
2. **小批次訓練**：batch_size < 8 時，加速效果不明顯
3. **數值不穩定**：出現 NaN 或 Inf 時
4. **精密科學計算**：例如軌道參數計算、凌日時間精確預測

```python
# 切換回 FP32 精度
model.float()  # 將模型轉回 FP32
with torch.no_grad():
    outputs = model(inputs.float())
```

### 外行星偵測特定優化

對於外行星凌日偵測任務：
- **特徵提取**：使用 FP32 以保持光曲線資料精度
- **神經網路訓練**：使用 BF16 加速訓練過程
- **BLS/TLS 週期搜尋**：維持 FP64 以獲得準確的週期估計
- **最終機率校準**：使用 FP32 確保可靠度分數準確

In [ ]:
# 持久化模型
os.makedirs("/content/model", exist_ok=True)
import joblib, json
joblib.dump(pipe, "/content/model/ranker.joblib")
with open("/content/model/feature_schema.json","w") as f:
    json.dump({"features": ['bls_period','bls_duration','bls_snr']}, f)
print("Saved to /content/model/")